In [ ]:
import psycopg2
import pandas as pd
from sshtunnel import SSHTunnelForwarder #Run pip install sshtunnel
from sqlalchemy.orm import sessionmaker #Run pip install sqlalchemy
from sqlalchemy import create_engine

tunnel = SSHTunnelForwarder(
    ('143.54.25.131', 22),
    ssh_username="atlas-oportunidades-srv01",
    ssh_password = "Mt0c5P!lzT",
    remote_bind_address=('localhost', 5432),
    local_bind_address=('localhost',8000), # could be any available port
)
# Start the tunnel
tunnel.start()

conn = create_engine(f'postgresql://atlas_adm:123456@{tunnel.local_bind_host}:{tunnel.local_bind_port}/atlasdb')

In [ ]:
dbConnection = conn.connect()
df_agencia = pd.read_sql("select nm_agencia from atlas_schema.tb_agencias", dbConnection)
dbConnection.close()

In [ ]:
#C:\Users\ricar\OneDrive\Documentos\Projetos\atlas_scripts\Data\_Census Tract\dictionary_census.xlsx
df = pd.read_excel(r"C:\Users\ricar\OneDrive\Documentos\Projetos\atlas_scripts\Data\_Census Tract\dictionary_census.xlsx")
colunas_manter = ['Agency']
df = df.loc[:, colunas_manter]
df = df.rename(columns={'Agency': 'nm_agencia'})
df = df.groupby('nm_agencia').count().reset_index()

df = df.drop_duplicates().merge(df_agencia.drop_duplicates(), on=df_agencia.columns.to_list(), how='left', indicator=True)
df = df.loc[df._merge=='left_only',df.columns!='_merge']
print(df)

if len(df) > 0:
    dbConnection = conn.connect()
    df.to_sql(    
        "tb_agencias",
        dbConnection,
        schema="atlas_schema",
        if_exists='append',
        index = False
    )
    dbConnection.close()
else:
    print('sem linhas de agencia a inserir')

In [ ]:
dbConnection = conn.connect()
df_formato = pd.read_sql("select nm_formato from atlas_schema.tb_formato", dbConnection)
dbConnection.close()

df = pd.read_excel(r"C:\Users\ricar\OneDrive\Documentos\Projetos\atlas_scripts\Data\_Census Tract\dictionary_census.xlsx")
colunas_manter = ['Format']
df = df.loc[:, colunas_manter]
df = df.rename(columns={'Format': 'nm_formato'})
df = df.groupby('nm_formato').count().reset_index()

df = df.drop_duplicates().merge(df_formato.drop_duplicates(), on=df_formato.columns.to_list(), how='left', indicator=True)
df = df.loc[df._merge=='left_only',df.columns!='_merge']
print(df)

if len(df) > 0:
    dbConnection = conn.connect()
    df.to_sql(    
        "tb_formato",
        dbConnection,
        schema="atlas_schema",
        if_exists='append',
        index = False
    )
    dbConnection.close()
else:
    print('sem linhas de formato a inserir')



In [ ]:
dbConnection = conn.connect()
df_unidade = pd.read_sql("select nm_unidade from atlas_schema.tb_unidades", dbConnection)
dbConnection.close()

df = pd.read_excel(r"C:\Users\ricar\OneDrive\Documentos\Projetos\atlas_scripts\Data\_Census Tract\dictionary_census.xlsx")
colunas_manter = ['Unit of analysis']
df = df.loc[:, colunas_manter]
df = df.rename(columns={'Unit of analysis': 'nm_unidade'})
df = df.groupby('nm_unidade').count().reset_index()

df = df.drop_duplicates().merge(df_unidade.drop_duplicates(), on=df_unidade.columns.to_list(), how='left', indicator=True)
df = df.loc[df._merge=='left_only',df.columns!='_merge']
print(df)

if len(df) > 0:
    dbConnection = conn.connect()
    df.to_sql(    
        "tb_unidades",
        dbConnection,
        schema="atlas_schema",
        if_exists='append',
        index = False
    )
    dbConnection.close()
else:
    print('sem linhas de unidade a inserir')

In [ ]:
dbConnection = conn.connect()
df_class = pd.read_sql("select nm_classificacao_pt, nm_classificacao_en from atlas_schema.tb_classificacao", dbConnection)
dbConnection.close()

df = pd.read_excel(r"C:\Users\ricar\OneDrive\Documentos\Projetos\atlas_scripts\Data\_Census Tract\dictionary_census.xlsx")
colunas_manter = ['Classificação', 'Classification']
df = df.loc[:, colunas_manter]
df = df.rename(columns={'Classificação': 'nm_classificacao_pt', 'Classification' : 'nm_classificacao_en'})
df = df.groupby(['nm_classificacao_pt', 'nm_classificacao_en']).count().reset_index()

df = df.drop_duplicates().merge(df_class.drop_duplicates(), on=df_class.columns.to_list(), how='left', indicator=True)
df = df.loc[df._merge=='left_only',df.columns!='_merge']
print(df)

if len(df) > 0:
    dbConnection = conn.connect()
    df.to_sql(    
        "tb_classificacao",
        dbConnection,
        schema="atlas_schema",
        if_exists='append',
        index = False
    )
    dbConnection.close()
else:
    print('sem linhas de classificacao a inserir')

In [ ]:
dbConnection = conn.connect()
df_agencia = pd.read_sql("select * from atlas_schema.tb_agencias", dbConnection)
df_unidade = pd.read_sql("select * from atlas_schema.tb_unidades", dbConnection)
df_formato = pd.read_sql("select * from atlas_schema.tb_formato", dbConnection)
df_classificacao = pd.read_sql("select * from atlas_schema.tb_classificacao", dbConnection)
dbConnection.close()

In [ ]:
df = pd.read_excel(r"C:\Users\ricar\OneDrive\Documentos\Projetos\atlas_scripts\Data\_Census Tract\dictionary_census.xlsx")
colunas_manter = ['Agency', 'Name', 'Description', 'Descrição', 'Label', 'Rótulo', 'Unit of analysis', 'Format', 'Classificação']
df = df.loc[:, colunas_manter]
df = df.rename(
    columns={
        'Name': 'cd_nm_coluna', 
        'Agency': 'nm_agencia',
        'Description': 'nm_descricao_en', 
        'Descrição': 'nm_descricao_pt', 
        'Label': 'nm_label_en', 
        'Rótulo': 'nm_label_pt', 
        'Unit of analysis': 'nm_unidade', 
        'Format': 'nm_formato', 
        'Classificação': 'nm_classificacao_pt'
    }
)

df_final = pd.merge(df, df_agencia, on='nm_agencia', how='left')
df_final = pd.merge(df_final, df_unidade, on='nm_unidade', how='left')
df_final = pd.merge(df_final, df_formato, on='nm_formato', how='left')
df_final = pd.merge(df_final, df_classificacao, on='nm_classificacao_pt', how='left')

colunas_manter = ['cd_nm_coluna', 'nm_descricao_en', 'nm_descricao_pt', 'nm_label_en', 'nm_label_pt', 'cd_agencia', 'cd_unidade', 'cd_formato', 'cd_classificacao']
df_final = df_final.loc[:, colunas_manter]

dbConnection = conn.connect()
df_final.to_sql(    
    "tb_dicionario_census",
    dbConnection,
    schema="atlas_schema",
    if_exists='append',
    index = False
)
dbConnection.close()

In [ ]:
df = pd.read_excel(
    r"C:\Users\ricar\OneDrive\Documentos\Projetos\atlas_scripts\Data\_Census Tract\Dados Censitarios.xlsx", 
    sheet_name="dados"
)
colunas_manter = ['Cod_setor']
df = df.melt(
    id_vars=['Cod_setor'],
    var_name='cd_nm_coluna',
    value_name='vl_por_cd'
).sort_values(by='Cod_setor')

df = df.rename(columns={'Cod_setor':'cd_setor'})

In [ ]:
dbConnection = conn.connect()
df.to_sql(    
    "tb_cod_valor_census",
    dbConnection,
    schema="atlas_schema",
    if_exists='append',
    index = False,
    chunksize=1000000
)
dbConnection.close()